generate nyctaxistream data asynchronously

In [2]:
import time, os
from datetime import datetime, timedelta
from utils.generate_data import generate_nyctaxistream_async,generate_locationid_temperature_async
from utils.kafka import write_to_kafka_async,list_kafka_topics,delete_kafka_topic,create_kafka_topic,close_kafka_admin,clean_reset_kafka_topics
from utils.spark import read_kafka,read_spark_parquet,write_spark_stream
import asyncio
from delta.tables import DeltaTable
from pyspark.sql.functions import to_timestamp
from utils import Startup
from types import SimpleNamespace
from utils.kafka import write_to_kafka_async,list_kafka_topics,delete_kafka_topic,create_kafka_topic,close_kafka_admin,clean_reset_kafka_topics


In [3]:
num_events = 100 #generates "num_events" events with random inputs except the fields which are specified foe each combination provided as input
latency = 0 # latency (seconds) between each event

# example
# {"dispatching_base_num":"B03494","PULocationID":100,"DOLocationID":100,"pickup_datetime":"2025-01-01 00:12:56","dropoff_datetime":"2025-01-01 00:26:41"} 

# write multiple event streams simultaneously
events = [
{"dispatching_base_num":"B03494","PULocationID":100,"DOLocationID":100,} 
,{"dispatching_base_num":"B03496","PULocationID":120,"DOLocationID":150,"pickup_datetime":"2025-01-01 00:15:56","dropoff_datetime":"2025-01-01 01:00:12"}
]

# firing multiple kafka events simultaneously
tasks = []

for event in events:
    tasks.append(asyncio.create_task(write_to_kafka_async(topic = "nyctaxistream", 
    message_generator=generate_nyctaxistream_async(num_events=num_events,latency=latency,**event))))
await asyncio.gather(*tasks)

Generated: {'hvfhs_license_num': 'HV00004', 'dispatching_base_num': 'B03494', 'PULocationID': 100, 'DOLocationID': 100, 'pickup_datetime': '2025-01-01 00:00:50', 'dropoff_datetime': '2025-01-01 00:03:23'}
Generated: {'hvfhs_license_num': 'HV00004', 'dispatching_base_num': 'B03496', 'PULocationID': 120, 'DOLocationID': 150, 'pickup_datetime': '2025-01-01 00:15:56', 'dropoff_datetime': '2025-01-01 01:00:12'}
Generated: {'hvfhs_license_num': 'HV00001', 'dispatching_base_num': 'B03494', 'PULocationID': 100, 'DOLocationID': 100, 'pickup_datetime': '2025-01-01 00:08:01', 'dropoff_datetime': '2025-01-01 00:21:04'}
Generated: {'hvfhs_license_num': 'HV00004', 'dispatching_base_num': 'B03496', 'PULocationID': 120, 'DOLocationID': 150, 'pickup_datetime': '2025-01-01 00:15:56', 'dropoff_datetime': '2025-01-01 01:00:12'}
Generated: {'hvfhs_license_num': 'HV00003', 'dispatching_base_num': 'B03494', 'PULocationID': 100, 'DOLocationID': 100, 'pickup_datetime': '2025-01-01 00:23:39', 'dropoff_datetime'

[None, None]